# 1. Installing libraries

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multipl

In [ ]:
# Warning Controls
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# for setting up API keys

import os

os.environ["OPENAI_API_KEY"]=""   # by generating keys in OPEN_AI
os.environ["SERPER_API_KEY"]=""   #from SERPER_AI

#set additional environment variable
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [ ]:
# Collect user inputs
print("Please provide the following inputs to analyze your stock:\n")

stock = input("Enter the stock you want to analyze (e.g., AAPL, TSLA): ").strip().upper()
initial_capital = float(input("Enter your initial capital (in USD): "))
risk_tolerance = input("Enter your risk tolerance (Low/Medium/High): ").strip().capitalize()
trading_strategy = input("Enter your trading strategy preference (e.g., Day Trading, Swing Trading, Long-term Investing): ").strip()

# Store inputs in a dictionary
financial_trading_inputs = {
    'stock': stock,
    'initial_capital': initial_capital,
    'risk_tolerance': risk_tolerance,
    'trading_strategy': trading_strategy
}

Please provide the following inputs to analyze your stock:

Enter the stock you want to analyze (e.g., AAPL, TSLA): TSLA
Enter your initial capital (in USD): 1000
Enter your risk tolerance (Low/Medium/High): Medium
Enter your trading strategy preference (e.g., Day Trading, Swing Trading, Long-term Investing): Day Trading


In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# 2. Creating Agents

In [ ]:
# Stock Performance Analyst
stock_performance_analyst = Agent(
    role="Stock Performance Analyst",
    goal=f"Analyze live market data for {financial_trading_inputs['stock']} and predict future performance for a {financial_trading_inputs['risk_tolerance']} risk tolerance and {financial_trading_inputs['initial_capital']} USD investment.",
    backstory=(
        "With over a decade of experience in financial markets, this agent specializes "
        "in analyzing stock performance using advanced quantitative models. "
        "It has a proven track record of identifying growth stocks and predicting "
        "market trends with high accuracy. Its expertise lies in combining technical "
        "analysis, fundamental analysis, and machine learning to deliver actionable insights."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# Portfolio Risk Strategist
portfolio_risk_strategist = Agent(
    role="Portfolio Risk Strategist",
    goal=f"Evaluate the risks associated with investing in {financial_trading_inputs['stock']} for a {financial_trading_inputs['risk_tolerance']} risk tolerance and recommend mitigation strategies.",
    backstory=(
        "A seasoned risk management expert, this agent has worked with top hedge funds "
        "and investment banks. It specializes in identifying portfolio risks, conducting "
        "stress tests, and developing risk mitigation strategies. Its expertise includes "
        "Value at Risk (VaR) modeling, scenario analysis, and portfolio optimization."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# Regulatory Compliance Officer
regulatory_compliance_officer = Agent(
    role="Regulatory Compliance Officer",
    goal=f"Ensure {financial_trading_inputs['stock']} complies with all legal and regulatory standards.",
    backstory=(
        "This agent is a certified compliance professional with extensive knowledge of global "
        "financial regulations, including SEC, FINRA, and MiFID II. It has a strong background "
        "in auditing trading activities, ensuring adherence to compliance frameworks, and "
        "preventing regulatory violations. Its meticulous approach ensures that all investments are legally sound."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]  # Added a compliance check tool
)

# Sentiment Analysis Specialist
sentiment_analysis_specialist = Agent(
    role="Sentiment Analysis Specialist",
    goal=f"Analyze news, social media, and other textual data to gauge market sentiment for {financial_trading_inputs['stock']}. Consider news impact.",
    backstory=(
        "This agent is an expert in natural language processing (NLP) and sentiment analysis. "
        "It has worked with leading financial institutions to analyze unstructured data "
        "from news articles, social media, and earnings calls. Its ability to identify "
        "sentiment shifts and predict market reactions makes it a valuable asset for decision-making."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
)

# Investment Decision Advisor
investment_decision_advisor = Agent(
    role="Investment Decision Advisor",
    goal=f"Provide a final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']} based on insights from all analysts. Consider a {financial_trading_inputs['risk_tolerance']} risk tolerance, {financial_trading_inputs['initial_capital']} USD investment, and trading strategy preference: {financial_trading_inputs['trading_strategy']}.",
    backstory=(
        "This agent is a senior investment advisor with over 15 years of experience in "
        "portfolio management. It excels at synthesizing complex data from multiple "
        "sources to make well-informed investment decisions. Its ability to balance risk "
        "and reward, combined with a deep understanding of market dynamics, ensures that "
        "its recommendations are both strategic and actionable."
    ),
    verbose=True,
    allow_delegation=False,
)

# 3. Creating Tasks

In [ ]:
from crewai import Task

# Stock Performance Analysis Task
stock_performance_task = Task(
    description=(
        f"Analyze live market data for {financial_trading_inputs['stock']} using technical and fundamental analysis. "
        f"Evaluate key performance indicators such as price trends, volume, moving averages, RSI, and financial metrics. "
        f"Predict the stock’s future performance for a {financial_trading_inputs['risk_tolerance']} risk tolerance and {financial_trading_inputs['initial_capital']} USD investment."
    ),
    agent=stock_performance_analyst,
    expected_output=(
        f"A detailed report on {financial_trading_inputs['stock']}'s performance, including: "
        "- Current price trends and technical indicators. "
        "- Growth potential based on fundamental analysis. "
        "- Predictions for short-term, mid-term, and long-term performance."
    ),
)

# Risk Assessment Task
risk_assessment_task = Task(
    description=(
        f"Evaluate the risks associated with investing in {financial_trading_inputs['stock']}. "
        f"Conduct stress tests, scenario analysis, and risk modeling for a {financial_trading_inputs['risk_tolerance']} risk tolerance. "
        "Assess the stock’s volatility, liquidity, and exposure to market risks. "
        "Provide a comprehensive risk assessment and recommend mitigation strategies."
    ),
    agent=portfolio_risk_strategist,
    expected_output=(
        f"A risk assessment report for {financial_trading_inputs['stock']}, including: "
        "- Identified risks (e.g., market risk, liquidity risk, regulatory risk). "
        "- Stress test results and worst-case scenarios. "
        "- Mitigation strategies (e.g., diversification, hedging)."
    ),
)

# Compliance Check Task
compliance_check_task = Task(
    description=(
        f"Ensure {financial_trading_inputs['stock']} complies with all legal and regulatory standards. "
        "Audit the stock’s trading activities, financial disclosures, and corporate governance practices. "
        "Check for any ongoing investigations or regulatory actions against the company."
    ),
    agent=regulatory_compliance_officer,
    expected_output=(
        f"A compliance report for {financial_trading_inputs['stock']}, including: "
        "- Regulatory adherence status. "
        "- Any compliance issues or red flags. "
        "- Recommendations for corrective actions (if needed)."
    ),
)

# Sentiment Analysis Task
sentiment_analysis_task = Task(
    description=(
        f"Analyze market sentiment for {financial_trading_inputs['stock']} using news articles, social media posts, and earnings call transcripts. "
        "Consider news impact. "
        "Use natural language processing (NLP) to identify sentiment trends (positive, negative, or neutral). "
        "Gauge public perception and predict how sentiment might impact the stock’s performance."
    ),
    agent=sentiment_analysis_specialist,
    expected_output=(
        f"A sentiment analysis report for {financial_trading_inputs['stock']}, including: "
        "- Key sentiment trends (e.g., bullish, bearish). "
        "- Public perception of recent events. "
        "- Potential impact of sentiment on stock performance."
    ),
)

# Decision Making Task
decision_task = Task(
    description=(
        f"Provide a final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']} based on insights from all analysts. "
        f"Consider a {financial_trading_inputs['risk_tolerance']} risk tolerance, {financial_trading_inputs['initial_capital']} USD investment, and trading strategy preference: {financial_trading_inputs['trading_strategy']}. "
        "Synthesize performance, risk, compliance, and sentiment analysis to make a well-informed recommendation."
    ),
    agent=investment_decision_advisor,
    expected_output=(
        f"A final decision (Buy, Hold, or Sell) for {financial_trading_inputs['stock']}, including: "
        "- Detailed reasoning for the decision. "
        "- Supporting data from all analyses. "
        "- Recommendations for next steps (e.g., entry/exit points, portfolio adjustments)."
    ),
)

# 4. Defining and Running Crew

In [ ]:
from IPython.display import display, Markdown
from crewai import Crew

# Define the crew
financial_trading_crew = Crew(
    agents=[
        stock_performance_analyst,
        portfolio_risk_strategist,
        regulatory_compliance_officer,
        sentiment_analysis_specialist,
        investment_decision_advisor,
    ],
    tasks=[
        stock_performance_task,
        risk_assessment_task,
        compliance_check_task,
        sentiment_analysis_task,
        decision_task,
    ],
    verbose=True,
)

# Run the crew
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Display the result
if result:
    display(Markdown(result))
else:
    print("Failed to generate the result")

 [DEBUG]: == Working Agent: Stock Performance Analyst
 [INFO]: == Starting Task: Analyze live market data for TSLA using technical and fundamental analysis. Evaluate key performance indicators such as price trends, volume, moving averages, RSI, and financial metrics. Predict the stock’s future performance for a Medium risk tolerance and 1000.0 USD investment.


> Entering new CrewAgentExecutor chain...
Action: Search the internet  
Action Input: {"search_query": "TSLA stock current price trends technical indicators fundamentals October 2023"}   


Search results: Title: Tesla, Inc. Common Stock (TSLA) Historical Quotes - Nasdaq
Link: https://www.nasdaq.com/market-activity/stocks/tsla/historical
Snippet: Find the latest historical data for Tesla, Inc. Common Stock (TSLA) at Nasdaq.com. View historical data in a monthly, bi-annual, or yearly format.
---
Title: Watch These Tesla Stock Price Levels After Post-Earnings Surge
Link: https://www.investopedia.com/tesla-stock-price-levels-to-wat

**Final Decision for TSLA: Hold**

**Reasoning for the Decision:**

1. **Current Market Sentiment**: The sentiment around Tesla is currently bearish due to regulatory scrutiny, particularly with the investigations into its self-driving technology. This has led to increased caution among investors, reflected in negative media coverage and social media discussions. Given that sentiment plays a crucial role in day trading, this bearish sentiment indicates potential short-term volatility, which could affect trading outcomes.

2. **Mixed Earnings Performance**: Tesla's Q3 2023 earnings report showed impressive growth in its Energy business and an increase in vehicle deliveries, yet the earnings per share fell short of analysts' expectations. The initial post-earnings surge of approximately 12% was countered by concerns about declining profit margins. This mixed performance suggests uncertainty about short-term price movements, making it less favorable for aggressive day trading.

3. **Potential for Future Upside**: While the current sentiment is cautious, there is optimism regarding Tesla's future growth potential, especially related to advancements in AI and self-driving features. If Tesla can navigate its compliance challenges and improve profit margins, analysts predict significant upside in the longer term. Thus, holding the stock allows for potential gains if the situation improves.

4. **Risk Management**: Given the medium risk tolerance and a preference for day trading, it is prudent to hold TSLA at this juncture rather than buying or selling outright. The stock may experience volatility, which could present day trading opportunities, but the current compliance issues pose risks that could lead to downward price movements in the short term.

**Supporting Data from Analyses:**

- **Sentiment Analysis**: The overall bearish sentiment due to regulatory scrutiny and mixed earnings performance could lead to short-term price declines, while positive long-term outlooks provide a buffer against immediate risks.
  
- **Volatility Indicators**: The ongoing compliance issues, combined with mixed earnings sentiment, suggest potential short-term volatility that can be harnessed for day trading strategies.

- **Analyst Predictions**: While some analysts express optimism regarding future performance, it is essential to recognize that the current risks may outweigh the potential rewards in the immediate term.

**Recommendations for Next Steps:**

- **Monitor Regulatory Developments**: Keep a close eye on news regarding the NHTSA investigation and any further recalls, as these will significantly influence stock price movements and trading opportunities.

- **Watch for Earnings Reports**: Future earnings reports will be crucial in determining Tesla’s ability to recover its profit margins. Positive results could provide significant upward momentum.

- **Set Entry/Exit Points**: If holding for day trading, consider setting tight entry and exit points based on market trends and news alerts. For example:
  - **Entry Point**: If the price dips below $240, consider buying shares to capitalize on potential recovery.
  - **Exit Point**: If the stock rises above $260, consider selling to lock in profits, especially if negative news arises.

- **Portfolio Adjustments**: Given the mixed performance and potential ups and downs, consider maintaining a diversified portfolio to mitigate risk while holding TSLA.

In conclusion, holding TSLA allows for strategic positioning amidst current market volatility while keeping an eye on potential future gains as the company navigates its challenges.